<a href="https://colab.research.google.com/github/carlamaziero/Reconhecimento_padroes/blob/main/Aula7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.14


In [2]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00537/sobar-72.csv")
X, y = df.iloc[:, :6], df.iloc[:, -1]
X.shape, y.shape

((72, 6), (72,))

In [4]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=42)
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((54, 6), (18, 6), (54,), (18,))

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


for Algoritmo in [KNeighborsClassifier, GaussianNB, Perceptron, DecisionTreeClassifier, RandomForestClassifier]:
    modelo = Algoritmo()
    modelo.fit(X_tr, y_tr)
    y_pr = modelo.predict(X_te)
    hits = y_pr == y_te
    print(modelo, sum(hits)/len(hits))

KNeighborsClassifier() 0.7222222222222222
GaussianNB() 0.7777777777777778
Perceptron() 0.6111111111111112
DecisionTreeClassifier() 0.7222222222222222
RandomForestClassifier() 0.7777777777777778


In [6]:
from sklearn.ensemble import VotingClassifier
modelo = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('naivebayes', GaussianNB()),
    ('perceptron', Perceptron()),
    ('arvore', DecisionTreeClassifier()),
    ('randomforest', RandomForestClassifier(random_state=42))
])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)

0.7222222222222222

In [7]:
from sklearn.ensemble import StackingClassifier

modelo = StackingClassifier([
    ('knn', KNeighborsClassifier()),
    ('naivebayes', GaussianNB()),
    ('perceptron', Perceptron()),
    ('arvore', DecisionTreeClassifier()),
    ('randomforest', RandomForestClassifier(random_state=42))
])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)

0.7222222222222222

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('naivebayes', GaussianNB()),
    ('perceptron', Perceptron()),
    ('arvore', DecisionTreeClassifier()),
    ('randomforest', RandomForestClassifier(random_state=42))
])

modelo = StackingClassifier([
    ('voting', voting),
    ('extratrees', ExtraTreesClassifier(random_state=42))

])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)

0.6666666666666666

In [10]:
from sklearn.model_selection import GridSearchCV
import numpy as np

par_knn = {'n_neighbors': [3,5,7]}
gs_knn = GridSearchCV(KNeighborsClassifier(), par_knn)

par_nb = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_naivebayes = GridSearchCV(GaussianNB(), par_nb)

par_per = {'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3]}
gs_perceptron = GridSearchCV(Perceptron(), par_per)

par_arvore = {'max_depth': [10, 15, 25]}
gs_arvore = GridSearchCV(DecisionTreeClassifier(), par_arvore)

par_randomforest = {'max_depth': [10, 15, 25], 'n_estimators': [100, 150, 200]}
gs_randomforest = GridSearchCV(RandomForestClassifier(random_state=42), par_randomforest)

voting = VotingClassifier([
    ('gs_knn', gs_knn),
    ('gs_naivebayes', gs_naivebayes),
    ('gs_perceptron', gs_perceptron),
    ('gs_arvore', gs_arvore),
    ('gs_randomforest', gs_randomforest)
])

modelo = StackingClassifier([
    ('voting', voting),
    ('extratrees', ExtraTreesClassifier(random_state=42))
])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)

0.6111111111111112